# RNN을 이용한 텍스트 생성
- '경마장에 있는 말이 뛰고 있다'
- '그의 말이 법이다'
- '가는 말이 고와야 오는 말이 곱다'

## 1. 데이터 전처리

In [1]:
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
seed = 2020
np.random.seed(seed)

In [3]:
# 3개의 문장을 변수에 저장
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [5]:
# 객체 t에서 사용가능한 method 및 attribute
dir(t)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_keras_api_names',
 '_keras_api_names_v1',
 'char_level',
 'document_count',
 'filters',
 'fit_on_sequences',
 'fit_on_texts',
 'get_config',
 'index_docs',
 'index_word',
 'lower',
 'num_words',
 'oov_token',
 'sequences_to_matrix',
 'sequences_to_texts',
 'sequences_to_texts_generator',
 'split',
 'texts_to_matrix',
 'texts_to_sequences',
 'texts_to_sequences_generator',
 'to_json',
 'word_counts',
 'word_docs',
 'word_index']

In [6]:
t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [7]:
# 단어 집합 크기 설정
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야함 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [8]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [9]:
sequences = []
for line in text.split('\n'): # \n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [10]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [11]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
max_len = max(len(s) for s in sequences) 
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 6


In [12]:
# 전체 샘플의 길이를 6(가장 긴 샘플의 길이)으로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [13]:
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]])

In [14]:
# 각 샘플의 마지막 단어를 레이블로 분리
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [15]:
X[:5]

array([[0, 0, 0, 0, 2],
       [0, 0, 0, 2, 3],
       [0, 0, 2, 3, 1],
       [0, 2, 3, 1, 4],
       [0, 0, 0, 0, 6]])

In [16]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [17]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

## 2. 모델 설계 및 학습

In [18]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, SimpleRNN

In [19]:
# 각 단어의 임베딩 벡터는 10차원, 은닉 상태 크기는 32
embedding = Embedding(vocab_size, 10, input_length=max_len-1) # 레이블을 분리하였으므로 X의 길이는 5
rnn = SimpleRNN(32)
output = Dense(vocab_size, activation='softmax')

In [20]:
model = Sequential()
model.add(embedding)
model.add(rnn)
model.add(output)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 10)             120       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                1376      
_________________________________________________________________
dense_1 (Dense)              (None, 12)                396       
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [22]:
history = model.fit(X, y, epochs=200, verbose=0)
history.history['accuracy'][-1]

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1.0

## 3. 모델 검증

In [23]:
# 모델 검증용 문장을 생성하는 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

#### Divide and Conquer

In [24]:
enc = t.texts_to_sequences(['경마장에'])[0]
enc = pad_sequences([enc], maxlen=5, padding='pre')
enc

array([[0, 0, 0, 0, 2]])

In [25]:
result = model.predict_classes(enc, verbose=0)
result

array([3], dtype=int64)

In [26]:
for word, index in t.word_index.items(): 
    if index == result: 
        break 
print(word)

있는


In [27]:
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 3번 예측
print(sentence_generation(model, t, '경마장에', 3))

경마장에 있는 말이 뛰고


In [28]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [29]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다


## Model 변화

### model2) Embedding Vector 차원=2, RNN 노드 수=32

In [30]:
model2 = Sequential()
model2.add(Embedding(vocab_size, 2, input_length=max_len-1))
model2.add(SimpleRNN(32))
model2.add(Dense(vocab_size, activation='softmax'))
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5, 2)              24        
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 32)                1120      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                396       
Total params: 1,540
Trainable params: 1,540
Non-trainable params: 0
_________________________________________________________________


In [31]:
model2.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [32]:
history2 = model2.fit(X, y, epochs=200, verbose=0)
history2.history['accuracy'][-1]

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.8181818

In [33]:
print(sentence_generation(model2, t, '경마장에', 3))
print(sentence_generation(model2, t, '그의', 2))
print(sentence_generation(model2, t, '가는', 5))

경마장에 말이 말이 뛰고
그의 말이 고와야
가는 말이 고와야 오는 말이 곱다


### model3) Embedding Vector 차원=4, RNN 노드 수=32

In [34]:
model3 = Sequential()
model3.add(Embedding(vocab_size, 4, input_length=max_len-1))
model3.add(SimpleRNN(32))
model3.add(Dense(vocab_size, activation='softmax'))
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 4)              48        
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                1184      
_________________________________________________________________
dense_3 (Dense)              (None, 12)                396       
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


In [35]:
model3.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [36]:
history3 = model3.fit(X, y, epochs=200, verbose=0)
history3.history['accuracy'][-1]

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.90909094

In [37]:
print(sentence_generation(model3, t, '경마장에', 3))
print(sentence_generation(model3, t, '그의', 2))
print(sentence_generation(model3, t, '가는', 5))

경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다


### model4) Embedding Vector 차원=4, RNN 노드 수=24

In [38]:
model4 = Sequential()
model4.add(Embedding(vocab_size, 4, input_length=max_len-1))
model4.add(SimpleRNN(24))
model4.add(Dense(vocab_size, activation='softmax'))
model4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 5, 4)              48        
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 24)                696       
_________________________________________________________________
dense_4 (Dense)              (None, 12)                300       
Total params: 1,044
Trainable params: 1,044
Non-trainable params: 0
_________________________________________________________________


In [39]:
model4.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [40]:
history4 = model4.fit(X, y, epochs=200, verbose=0)
history4.history['accuracy'][-1]

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.90909094

In [41]:
print(sentence_generation(model4, t, '경마장에', 3))
print(sentence_generation(model4, t, '그의', 2))
print(sentence_generation(model4, t, '가는', 5))

경마장에 말이 법이다 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다


### model5) Embedding Vector 차원=6, RNN 노드 수=24

In [42]:
model5 = Sequential()
model5.add(Embedding(vocab_size, 6, input_length=max_len-1))
model5.add(SimpleRNN(24))
model5.add(Dense(vocab_size, activation='softmax'))
model5.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 5, 6)              72        
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 24)                744       
_________________________________________________________________
dense_5 (Dense)              (None, 12)                300       
Total params: 1,116
Trainable params: 1,116
Non-trainable params: 0
_________________________________________________________________


In [43]:
model5.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [44]:
history5 = model5.fit(X, y, epochs=200, verbose=0)
history5.history['accuracy'][-1]

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1.0

In [45]:
print(sentence_generation(model5, t, '경마장에', 3))
print(sentence_generation(model5, t, '그의', 2))
print(sentence_generation(model5, t, '가는', 5))

경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다


### model6) Embedding Vector 차원=4, RNN 노드 수=28

In [46]:
model6 = Sequential()
model6.add(Embedding(vocab_size, 4, input_length=max_len-1))
model6.add(SimpleRNN(28))
model6.add(Dense(vocab_size, activation='softmax'))
model6.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 5, 4)              48        
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 28)                924       
_________________________________________________________________
dense_6 (Dense)              (None, 12)                348       
Total params: 1,320
Trainable params: 1,320
Non-trainable params: 0
_________________________________________________________________


In [47]:
model6.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [48]:
history6 = model6.fit(X, y, epochs=200, verbose=0)
history5.history['accuracy'][-1]

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1.0

In [49]:
print(sentence_generation(model6, t, '경마장에', 3))
print(sentence_generation(model6, t, '그의', 2))
print(sentence_generation(model6, t, '가는', 5))

경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
